In [47]:
%%html

<style>
img[alt="_"]{
    max-width: 500px;
    display: inline-block;
}
</style>

### Python 동시성 프로그래밍

- http://hamait.tistory.com/752?category=79136
- https://soooprmx.com/archives/6882
- http://hurderella.tistory.com/86

싱글스레드
> - http://pythonstudy.xyz/python/article/24-%EC%93%B0%EB%A0%88%EB%93%9C-Thread
> - http://highthroughput.org/wp/cb-1136/
> - https://yinjae.wordpress.com/2012/04/02/python-thread/

병렬처리
> - 멀티스레드 X (GIL)
> - 멀티 프로세스 O

계산비용
> - CPU BOUND
> - I/O BOUND

In [48]:
import time
import asyncio

# Synchronous

In [49]:
def is_prime(x):
    
    return not any(x // i == x / i for i in range(2, x - 1))


def highest_prime_below(x):
    
    print("run function for %d" % x)
    
    for y in range(x-1, 1, -1):
        if is_prime(y):
            print("Highest prime below %d is %d" % (x, y))
            break
        time.sleep(0.05)
    else:
        print("%d is smallest prime" % x)
    return None


def main():
    
    start = time.time()
    highest_prime_below(100000)
    highest_prime_below(10000)
    highest_prime_below(1000)
    highest_prime_below(2)
    print(time.time() - start)
    
    
if __name__ == "__main__":
    main()

run function for 100000
Highest prime below 100000 is 99991
run function for 10000
Highest prime below 10000 is 9973
run function for 1000
Highest prime below 1000 is 997
run function for 2
2 is smallest prime
1.8633947372436523


# Asynchronous

In [50]:
async def highest_prime_below(x):

    print("run function for %d" % x)
    
    for y in range(x-1, 1, -1):
        if is_prime(y):
            print("Highest prime below %d is %d" % (x, y))
            break
        await asyncio.sleep(0.05)
    else:
        print("%d is smallest prime" % x)
    return None


async def main():
    
    start = time.time()
    
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000),
        highest_prime_below(2)])
    
    print(time.time() - start)
    

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

run function for 1000
run function for 10000
run function for 2
2 is smallest prime
run function for 100000
Highest prime below 1000 is 997
Highest prime below 100000 is 99991
Highest prime below 10000 is 9973
1.3283908367156982


# CPU-BOUND EXAMPLE

In [203]:
N = 100000000
D = 5

def function(from_, to):
    
    print(sum(range(from_, to)))

# 단순처리

In [204]:
start = time.time()
dvs = [[N // D * i, N // D * (i + 1)] for i in range(D)]
for dv in dvs:
    function(*dv)
print(time.time() - start)

199999990000000
599999990000000
999999990000000
1399999990000000
1799999990000000
3.049006700515747


# 비동기처리

In [206]:
async def each_get(from_, to):

    await loop.run_in_executor(None, function, from_, to)

    
async def coget():
    
    dvs = [[N // D * i, N // D * (i + 1)] for i in range(D)]
    await asyncio.gather(*[each_get(*dv) for dv in dvs])
    
start = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(coget())
record = time.time() - start
print()
print(record)

59999999000000099999999000000019999999000000017999999900000001399999990000000




3.244000196456909



# I/O-BOUND EXAMPLE

In [199]:
import requests

N = 5

def FUNCTION():
    requests.get("https://sites.google.com/a/chromium.org/chromedriver/getting-started")

# 단순처리

In [200]:
if __name__ == "__main__":

    start = time.time()
    for _ in range(N):
        FUNCTION()
    print(time.time() - start)

2.228257179260254


# 비동기처리

In [201]:
async def each_get():

    await loop.run_in_executor(None, FUNCTION)

    
async def coget():
    
    await asyncio.wait([each_get() for _ in range(N)])
    

if __name__ == "__main__":

    start = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(coget())
    record = time.time() - start
    time.sleep(0.1)
    print(record)

0.401519775390625


```python
# 속도 개선이 안됨..

N = 10
MB = 300

with open("data", "wb") as f:
    f.write(b"\x00" * 1000 * 1000 * MB)

def FUNCTION():
    with open("data") as f:
        f.read()
```

# Chatting Server

In [5]:
!pip install websockets

In [ ]:
import asyncio
import websockets
import time


async def handler(websocket, path):

    global connected
    connected.add(websocket)
    n_of_con = len(connected)
    
    while True:
        await asyncio.sleep(2)
        if len(connected) == n_of_con:
            for ws in connected:
                try:
                    message = str(len(connected))
                    await ws.send(message)
                except:
                    pass
            

if __name__ == "__main__":

    connected = set()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        websockets.serve(handler, '0.0.0.0', 5678))
    loop.run_forever()

# 멀티프로세스 (윈도우에서는 스크립트로 돌려야함!)

![_](img/1.png)

In [4]:
from multiprocessing import Process
import time
import os


def GO(N, D):

    list(range(N // D))
    print("parent process: {} | process id: {}".format(os.getppid(), os.getpid()))
    
    
if __name__=='__main__':

    N = 100000000
    D = os.cpu_count()
    
    start = time.time()

    processes = []

    for i in range(D):
        proc = Process(target=GO, args=(N, D))
        processes.append(proc)
    
    for proc in processes:
        proc.start()
    
    for proc in processes:
        proc.join()
        
    print(time.time() - start)

0.22705459594726562


# Pool

In [ ]:
from multiprocessing import Pool
import os


def f(x):
    
    print("process id: {} | value: {}".format(os.getpid(), x))
    return os.getpid()


if __name__ == '__main__':
        
    D = os.cpu_count()
    
    with Pool(D) as p:
        print(set(p.map(f, range(8))))

# Queue

In [14]:
from multiprocessing import Process, Queue
import time


def f1(q):
    
    data = list(range(10))
    q.put(data)

    
def f2(q):
    
    data = list(range(10, 20))
    data = q.put(data)

    
    
if __name__ == '__main__':
    
    q = Queue()
    
    p1 = Process(target=f1, args=(q,))
    p2 = Process(target=f2, args=(q,))
    p1.start()
    p2.start()

    p1.join()
    p2.join()
    
    print(q.get())
    print(q.get())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


# 암호화

In [14]:
from Crypto.Hash import SHA256
from base64 import b64encode, b64decode


if __name__ == "__main__":

    hashing = SHA256.new()
    hashing.update(b'seohasong')

    암호화된값 = hashing.hexdigest()
    쿠키값 = b64encode(암호화된값.encode("utf-8")).decode("utf-8")
    쿠키값디코딩 = b64decode(쿠키값).decode("utf-8")
    
    print("암호화된값: ", 암호화된값)
    print()
    print("쿠키값: ", 쿠키값)
    print()
    print("쿠키값디코딩: ", 쿠키값디코딩)

암호화된값:  29de565b5a62403b46790bac6efeb40a8f1f2161b03cd23b2fbdeeb96abd72d4

쿠키값:  MjlkZTU2NWI1YTYyNDAzYjQ2NzkwYmFjNmVmZWI0MGE4ZjFmMjE2MWIwM2NkMjNiMmZiZGVlYjk2YWJkNzJkNA==

쿠키값디코딩:  29de565b5a62403b46790bac6efeb40a8f1f2161b03cd23b2fbdeeb96abd72d4


# 세션 탈취

![_](img/2.png)

In [15]:
import requests

cookie = "remember_web_59ba36addc2b2f9401580f014c7f58ea4e30989d=eyJpdiI6Imd4bkNSYUUrbDBsbTdyOVFTRTBCdkE9PSIsInZhbHVlIjoiUmFvZ0ZLdmlWTGcrMzgxQ0U5d3lrdDFPRU1BS0h6dXUrMXlrb0pMdWZlWmdNXC9CaTJtOWtQZW1ka0ZxZ0ZNNHhPTjFZMzVNSnlOQVo2MTI1RWNleWxhUjRCaElhU1BJbmlyK1IraU1tajZVPSIsIm1hYyI6IjVjYjNiOTFhNTdkODE1ZjJiMTkwYzdlZGFkMGNjMjExMGUwNWJjMTg0OWZhZDExMzU2YTE1ZmFkMzNiMzY3NDAifQ"

headers = {
    "Cookie": cookie
}

json_data = requests.get("https://api.klue.kr/info/mypage", headers=headers).text
쿠키값디코딩 = b64decode(cookie.split("=")[-1] + "==").decode("utf-8")

print("json_data: ", json_data)
print()
print("쿠키값디코딩: ", 쿠키값디코딩)

json_data:  {"code":200,"data":{"id":"tisutoo","sc_id":"ku","sc_email":"tisutoo","phone_number":null,"coffee_bean":0,"points":10,"evaluation_count":24,"note_count":0,"purchase_note_count":0,"facebook":null,"google":null,"read_lec_eval_authority":true}}

쿠키값디코딩:  {"iv":"gxnCRaE+l0lm7r9QSE0BvA==","value":"RaogFKviVLg+381CE9wykt1OEMAKHzuu+1ykoJLufeZgM\/Bi2m9kPemdkFqgFM4xON1Y35MJyNAZ6125EceylaR4BhIaSPInir+R+iMmj6U=","mac":"5cb3b91a57d815f2b190c7edad0cc2110e05bc1849fad11356a15fad33b36740"}


# Headless Chrome

```bash
# alias chrome="/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome"
chrome --headless --disable-gpu --dump-dom https://klue.kr/ > index.html
python -m SimpleHTTPServer 1111
```

# 세션 탈취 + Selenium Webdriver + Headless Chrome

- https://sites.google.com/a/chromium.org/chromedriver/getting-started